In [1]:
import datasets

eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]
# for example in eval_set:
#     # generate here is a placeholder for your models generations
#     messages = [
#         {"role": "user", "content": example["instruction"]}
#     ]
#     example["output"] = generate(example["instruction"])
#     example["generator"] = "my_model" # name of your model

/home/lidong1/miniconda3/envs/eval/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for tatsu-lab/alpaca_eval contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/tatsu-lab/alpaca_eval
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [4]:
import json
# 定义保存文件的路径  
output_file = "./data/alpaca_eval_dataset.jsonl"  

# 将数据集保存为 JSONL 格式  
with open(output_file, 'w', encoding='utf-8') as f:  
    for record in eval_set:  
        json_record = json.dumps(record, ensure_ascii=False)  
        f.write(json_record + '\n')  

print(f"数据集已保存到 {output_file}")  

数据集已保存到 ./data/alpaca_eval_dataset.jsonl


In [32]:
eval_set[0]

{'instruction': 'What are the names of some famous actors that started their careers on Broadway?',
 'output': 'Some famous actors that started their careers on Broadway include: \n1. Hugh Jackman \n2. Meryl Streep \n3. Denzel Washington \n4. Julia Roberts \n5. Christopher Walken \n6. Anthony Rapp \n7. Audra McDonald \n8. Nathan Lane \n9. Sarah Jessica Parker \n10. Lin-Manuel Miranda',
 'generator': 'text_davinci_003',
 'dataset': 'helpful_base'}

In [4]:
template = "{{ '<|begin_of_text|>' }}{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ '<|start_header_id|>system<|end_header_id|>\n\n' + system_message + '<|eot_id|>' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<|start_header_id|>user<|end_header_id|>\n\n' + content + '<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n' }}{% elif message['role'] == 'assistant' %}{{ content + '<|eot_id|>' }}{% endif %}{% endfor %}"

In [1]:
from vllm import LLM, SamplingParams
from tqdm import tqdm
import datasets
template = "{{ '<|begin_of_text|>' }}{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ '<|start_header_id|>system<|end_header_id|>\n\n' + system_message + '<|eot_id|>' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<|start_header_id|>user<|end_header_id|>\n\n' + content + '<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n' }}{% elif message['role'] == 'assistant' %}{{ content + '<|eot_id|>' }}{% endif %}{% endfor %}"

sampling_params = SamplingParams(temperature=0.8, top_p=0.95)
model_name = "Meta-Llama-3.1-8B"
# Create an LLM.
llm = LLM(model=f"/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface/{model_name}")

tokenizer = llm.get_tokenizer()
# tokenizer.chat_template = template
if tokenizer.chat_template is None:
    tokenizer.chat_template = template
    print("tokenizer is None, use setted template")
else:
    print("use original template")

/home/lidong1/miniconda3/envs/eval/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-12 19:50:24,574	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING 08-12 19:50:27 arg_utils.py:766] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 08-12 19:50:27 config.py:820] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 08-12 19:50:27 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface/Meta-Llama-3.1-8B', speculative_config=None, tokenizer='/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface/Meta-Llama-3.1-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.41it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.16it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.10it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.40it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.31it/s]



INFO 08-12 19:51:51 model_runner.py:732] Loading model weights took 14.9888 GB
INFO 08-12 19:51:51 gpu_executor.py:102] # GPU blocks: 12313, # CPU blocks: 2048
INFO 08-12 19:51:54 model_runner.py:1024] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 08-12 19:51:54 model_runner.py:1028] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 08-12 19:52:07 model_runner.py:1225] Graph capturing finished in 13 secs.
tokenizer is None


In [1]:
from vllm import LLM, SamplingParams
from tqdm import tqdm
import datasets
template = "{{ '<|begin_of_text|>' }}{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ '<|start_header_id|>system<|end_header_id|>\n\n' + system_message + '<|eot_id|>' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<|start_header_id|>user<|end_header_id|>\n\n' + content + '<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n' }}{% elif message['role'] == 'assistant' %}{{ content + '<|eot_id|>' }}{% endif %}{% endfor %}"

sampling_params = SamplingParams(temperature=0.8, top_p=0.95)
# model_name = "Meta-Llama-3.1-8B-Instruct"
model_name = "Meta-Llama-3.1-8B"
# Create an LLM.
llm = LLM(model=f"/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface/{model_name}")

tokenizer = llm.get_tokenizer()
if tokenizer.chat_template is None:
    tokenizer.chat_template = template
    print("tokenizer is None, use setted template")
else:
    print("use original template")
# messages = tokenizer.apply_chat_template(messages, tokenize=False)
eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]

WARNING 08-12 23:31:48 arg_utils.py:766] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 08-12 23:31:48 config.py:820] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 08-12 23:31:48 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface/Meta-Llama-3.1-8B', speculative_config=None, tokenizer='/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface/Meta-Llama-3.1-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 08-12 23:33:01 model_runner.py:732] Loading model weights took 14.9888 GB
INFO 08-12 23:33:01 gpu_executor.py:102] # GPU blocks: 12313, # CPU blocks: 2048
INFO 08-12 23:33:04 model_runner.py:1024] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 08-12 23:33:04 model_runner.py:1028] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 08-12 23:33:17 model_runner.py:1225] Graph capturing finished in 13 secs.
tokenizer is None, use setted template


In [11]:
gen_kwargs_vllm = {
    "max_tokens": 2048,
    "top_p": 0.9,
    "top_k": 50,
    "temperature": 0.0,
    "repetition_penalty": 1.0,
}
gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|end_of_text|>")]
sampling_params = SamplingParams(**gen_kwargs_vllm)
sampling_params

SamplingParams(n=1, best_of=1, presence_penalty=0.0, frequency_penalty=0.0, repetition_penalty=1.0, temperature=0.0, top_p=1.0, top_k=-1, min_p=0.0, seed=None, use_beam_search=False, length_penalty=1.0, early_stopping=False, stop=[], stop_token_ids=[128001, 128001], include_stop_str_in_output=False, ignore_eos=False, max_tokens=1024, min_tokens=0, logprobs=None, prompt_logprobs=None, skip_special_tokens=True, spaces_between_special_tokens=True, truncate_prompt_tokens=None)

In [12]:
def convert_to_message(example):  
    messages = [{"role": "user", "content": example["instruction"]}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
eval_set = eval_set.map(convert_to_message)
# eval_set['messages']
outputs = llm.generate(eval_set['messages'], sampling_params)
outputs_text = [x.outputs[0].text for x in outputs]
eval_set = eval_set.remove_columns(["output"])  # Remove the existing 'output' column if it exists  
eval_set = eval_set.remove_columns(["messages"])
eval_set = eval_set.add_column("output", outputs_text)  
eval_set.to_json(f"{model_name}.jsonl", batch_size=128, num_proc=8)

Map:   0%|          | 0/805 [00:00<?, ? examples/s]

Processed prompts:  12%|█▏        | 94/805 [01:36<40:47,  3.44s/it, est. speed input: 46.73 toks/s, output: 228.97 toks/s] 

WARNING 08-12 23:52:17 scheduler.py:1099] Sequence group 1746 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=301


Processed prompts:  57%|█████▋    | 456/805 [04:04<18:29,  3.18s/it, est. speed input: 90.65 toks/s, output: 1218.88 toks/s] 

WARNING 08-12 23:54:44 scheduler.py:1099] Sequence group 2045 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=351


Processed prompts: 100%|██████████| 805/805 [05:32<00:00,  2.42it/s, est. speed input: 112.45 toks/s, output: 1859.61 toks/s]


In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("allenai/tulu-2-dpo-7b")
# tokenizer

tokenizer_config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [1]:
from vllm import LLM, SamplingParams
from tqdm import tqdm
import datasets
import json
# template = "{{ '<|begin_of_text|>' }}{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ '<|start_header_id|>system<|end_header_id|>\n\n' + system_message + '<|eot_id|>' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<|start_header_id|>user<|end_header_id|>\n\n' + content + '<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n' }}{% elif message['role'] == 'assistant' %}{{ content + '<|eot_id|>' }}{% endif %}{% endfor %}"
template = "{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ system_message + '\n' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ 'Human: ' + content + '\nAssistant:' }}{% elif message['role'] == 'assistant' %}{{ content + '<|end_of_text|>' + '\n' }}{% endif %}{% endfor %}"

# model_name = "Meta-Llama-3.1-8B-Instruct"
# model_name = "Meta-Llama-3.1-8B"

# model_name = "tulu_lora_sft_default_template_8b"
model_name = "tulu-2-dpo-7b"
# Create an LLM.
# llm = LLM(model=f"/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface/{model_name}")

llm = LLM(model=f"/mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints/{model_name}")
tokenizer = llm.get_tokenizer()
gen_kwargs_vllm = {
    "max_tokens": 2048,
    "top_p": 0.9,
    "top_k": 50,
    "temperature": 0.0,
    "repetition_penalty": 1.0,
}
if tokenizer.chat_template is None:
    tokenizer.chat_template = template
    # tokenizer.chat_template = tokenizer.chat_template.replace("<|eot_id|>", tokenizer.eos_token)
    # tokenizer.chat_template
    gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]
    print(f"tokenizer.chat_template: {tokenizer.chat_template}")
    print("tokenizer is None, use setted template")
else:
    gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|end_of_text|>")]
    print("use original template")
# messages = tokenizer.apply_chat_template(messages, tokenize=False)



sampling_params = SamplingParams(**gen_kwargs_vllm)

eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]

def convert_to_message(example):  
    messages = [{"role": "user", "content": example["instruction"]}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
eval_set = eval_set.map(convert_to_message)
# eval_set['messages']
outputs = llm.generate(eval_set['messages'], sampling_params)
outputs_text = [x.outputs[0].text for x in outputs]
eval_set = eval_set.remove_columns(["output"])  # Remove the existing 'output' column if it exists  
# eval_set = eval_set.remove_columns(["messages"])
eval_set = eval_set.add_column("output", outputs_text)  
def rename_generator(sample):
    sample['generator'] = f"{model_name}"
    return sample
eval_set = eval_set.map(rename_generator)
eval_set.to_json(f"{model_name}.jsonl", batch_size=128, num_proc=8)

## save data

export_dataset = eval_set
export_data_list = [dict(row) for row in export_dataset]
print(f"export data length {len(export_data_list)}")
with open(f'./data/{model_name}.json', 'w', encoding='utf-8') as f:   
    json.dump(export_data_list, f, ensure_ascii=False, indent=4)  

INFO 08-17 23:12:49 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='/mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints/tulu-2-dpo-7b', speculative_config=None, tokenizer='/mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints/tulu-2-dpo-7b', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=/mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints/tulu-2-dpo-7b, 

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


RuntimeError: Internal: could not parse ModelProto from /mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints/tulu-2-dpo-7b/tokenizer.model

In [2]:
eval_set[:5]

{'instruction': ['What are the names of some famous actors that started their careers on Broadway?',
  'How did US states get their names?',
  "Hi, my sister and her girlfriends want me to play kickball with them. Can you explain how the game is played, so they don't take advantage of me?",
  'What is some cool music from the 1920s?',
  'How do I wrap a present neatly?'],
 'generator': ['Meta-Llama-3.1-8B',
  'Meta-Llama-3.1-8B',
  'Meta-Llama-3.1-8B',
  'Meta-Llama-3.1-8B',
  'Meta-Llama-3.1-8B'],
 'dataset': ['helpful_base',
  'helpful_base',
  'helpful_base',
  'helpful_base',
  'helpful_base'],
 'messages': ['Human: What are the names of some famous actors that started their careers on Broadway?\nAssistant:',
  'Human: How did US states get their names?\nAssistant:',
  "Human: Hi, my sister and her girlfriends want me to play kickball with them. Can you explain how the game is played, so they don't take advantage of me?\nAssistant:",
  'Human: What is some cool music from the 1920s

In [3]:
import json
from transformers import AutoTokenizer
# model_name = "tulu_v2_8b_bsz64_default_template_dpo"
mytokenizer = AutoTokenizer.from_pretrained(f"/mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints/tulu-lora-sft-llama3-8b-base")
mytokenizer.eos_token_id
with open(f'data/{model_name}.json', 'r') as f:
    print(f"file path: data/{model_name}.json")
    data = json.load(f)
data
outputs = [x['output'] for x in data]
lengths = [len(mytokenizer.encode(x)) for x in outputs]
lengths
from collections import Counter
Counter(lengths)

file path: data/Meta-Llama-3.1-8B.json


Counter({2049: 669,
         6: 3,
         17: 3,
         71: 2,
         347: 2,
         168: 2,
         53: 2,
         982: 2,
         7: 2,
         387: 2,
         368: 2,
         380: 2,
         47: 2,
         19: 2,
         113: 2,
         14: 2,
         139: 1,
         120: 1,
         80: 1,
         525: 1,
         43: 1,
         1084: 1,
         109: 1,
         25: 1,
         750: 1,
         132: 1,
         1521: 1,
         563: 1,
         1269: 1,
         149: 1,
         138: 1,
         159: 1,
         12: 1,
         95: 1,
         326: 1,
         239: 1,
         406: 1,
         603: 1,
         155: 1,
         481: 1,
         1119: 1,
         541: 1,
         960: 1,
         49: 1,
         189: 1,
         91: 1,
         292: 1,
         477: 1,
         165: 1,
         697: 1,
         173: 1,
         427: 1,
         195: 1,
         46: 1,
         279: 1,
         60: 1,
         85: 1,
         225: 1,
         255: 1,
         1

In [35]:
## save data
import json
export_dataset = eval_set.select(range(10))
export_data_list = [dict(row) for row in export_dataset]
print(f"export data length {len(export_data_list)}")
with open(f'./data/{model_name}.json', 'w', encoding='utf-8') as f:  
    json.dump(export_data_list, f, ensure_ascii=False, indent=4)  

export data length 10


In [23]:
from transformers import AutoTokenizer
model_name = "Meta-Llama-3.1-8B"
mytokenizer = AutoTokenizer.from_pretrained(f"/mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints/tulu-lora-sft-llama3-8b-base")
mytokenizer.eos_token_id

128009

In [12]:
from transformers import AutoTokenizer
model_name = "Meta-Llama-3.1-8B-Instruct"
mytokenizer = AutoTokenizer.from_pretrained(f"/mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints/{model_name}")
mytokenizer.chat_template
messages = [{"role": "user", "content": "hello"}, {"role": "assistant", "content": "hi"}]  
output = mytokenizer.apply_chat_template(messages, tokenize=False)
print(output)


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

hello<|eot_id|><|start_header_id|>assistant<|end_header_id|>

hi<|eot_id|>


In [2]:
from transformers import AutoTokenizer
model_name = "tulu_lora_sft_default_template_8b"
mytokenizer = AutoTokenizer.from_pretrained(f"/mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints/{model_name}")
mytokenizer.chat_template
messages = [{"role": "user", "content": "hello"}, {"role": "assistant", "content": "hi"}]  
output = mytokenizer.apply_chat_template(messages, tokenize=False)
print(output)


Human: hello
Assistant:hi<|end_of_text|>



In [7]:
mytokenizer.chat_template

"{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ system_message + '\n' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ 'Human: ' + content + '\nAssistant:' }}{% elif message['role'] == 'assistant' %}{{ content + '<|end_of_text|>' + '\n' }}{% endif %}{% endfor %}"

In [6]:
with open("./default.jinja", 'w') as f:
    f.write(repr(mytokenizer.chat_template))


In [13]:
from transformers import AutoTokenizer
model_name = "tulu-dpo-llama3-8b-base"
mytokenizer = AutoTokenizer.from_pretrained(f"/mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints/{model_name}")
mytokenizer.chat_template
messages = [{"role": "user", "content": "hello"}, {"role": "assistant", "content": "hi"}]  
output = mytokenizer.apply_chat_template(messages, tokenize=False)
print(output)


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

hello<|eot_id|><|start_header_id|>assistant<|end_header_id|>

hi<|eot_id|>


In [14]:
from transformers import AutoTokenizer
model_name = "tulu_lora_sft_base_template_8b"
mytokenizer = AutoTokenizer.from_pretrained(f"/mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints/{model_name}")
mytokenizer.chat_template
messages = [{"role": "user", "content": "hello"}, {"role": "assistant", "content": "hi"}]  
output = mytokenizer.apply_chat_template(messages, tokenize=False)
print(output)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

hello<|end_of_text|><|start_header_id|>assistant<|end_header_id|>

hi<|end_of_text|>


In [38]:
from transformers import AutoTokenizer
model_name = "Meta-Llama-3.1-8B"
mytokenizer = AutoTokenizer.from_pretrained("/home/lidong1/jianglingjie/LLama-Factory/saves/tulu_v2_8b_test/lora/sft")
mytokenizer.chat_template
messages = [{"role": "user", "content": "hello"}, {"role": "assistant", "content": "yes"}]  
# messages = [{"role": "user", "content": "hello"}]  
output = mytokenizer.apply_chat_template(messages, tokenize=False)
print(output)

Human: hello
Assistant:yes<|end_of_text|>



In [45]:
import os
datapath = os.path.join('data', '/mnt/lingjiejiang/textual_aesthetics/data/tulu_v2.json')
datapath

'/mnt/lingjiejiang/textual_aesthetics/data/tulu_v2.json'

In [36]:
from transformers import AutoTokenizer
model_name = "Meta-Llama-3.1-8B"
mytokenizer = AutoTokenizer.from_pretrained("/home/lidong1/jianglingjie/LLama-Factory/saves/tulu_v2_8b_base_template/lora/sft")
mytokenizer.chat_template
messages = [{"role": "user", "content": "hello"}, {"role": "assistant", "content": "yes"}]  
output = mytokenizer.apply_chat_template(messages, tokenize=False)
print(output)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

hello<|end_of_text|><|start_header_id|>assistant<|end_header_id|>

yes<|end_of_text|>


In [3]:
from transformers import AutoTokenizer
model_name = "Meta-Llama-3.1-8B"
mytokenizer = AutoTokenizer.from_pretrained(f"/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface/{model_name}")
mytokenizer.eos_token_id

128001

In [4]:
import json

with open('data/tulu_lora_sft_default_template_8b.json', 'r') as f:
    data = json.load(f)
data
outputs = [x['output'] for x in data]
lengths = [len(mytokenizer.encode(x)) for x in outputs]
lengths
from collections import Counter
Counter(lengths)

Counter({1025: 17,
         193: 7,
         111: 6,
         299: 6,
         91: 6,
         80: 6,
         64: 6,
         267: 5,
         278: 5,
         85: 5,
         88: 5,
         309: 5,
         3: 5,
         98: 5,
         313: 5,
         108: 5,
         141: 4,
         227: 4,
         168: 4,
         218: 4,
         165: 4,
         62: 4,
         334: 4,
         68: 4,
         259: 4,
         276: 4,
         104: 4,
         89: 4,
         86: 4,
         250: 4,
         4: 4,
         252: 4,
         81: 4,
         17: 4,
         13: 4,
         11: 4,
         266: 3,
         174: 3,
         328: 3,
         344: 3,
         380: 3,
         101: 3,
         61: 3,
         52: 3,
         116: 3,
         315: 3,
         256: 3,
         97: 3,
         235: 3,
         290: 3,
         232: 3,
         314: 3,
         245: 3,
         335: 3,
         176: 3,
         264: 3,
         355: 3,
         272: 3,
         286: 3,
         291: 3,

In [19]:
lengths = [len(mytokenizer.encode(x)) for x in outputs]
lengths
from collections import Counter
Counter(lengths)

Counter({1025: 551,
         1024: 42,
         1023: 21,
         1027: 5,
         1026: 5,
         1019: 4,
         281: 3,
         727: 3,
         1022: 3,
         1029: 3,
         273: 2,
         644: 2,
         819: 2,
         1021: 2,
         552: 2,
         408: 2,
         776: 2,
         747: 2,
         833: 2,
         601: 2,
         949: 2,
         992: 2,
         718: 1,
         547: 1,
         606: 1,
         634: 1,
         544: 1,
         624: 1,
         805: 1,
         480: 1,
         586: 1,
         730: 1,
         607: 1,
         465: 1,
         826: 1,
         863: 1,
         283: 1,
         562: 1,
         351: 1,
         830: 1,
         635: 1,
         798: 1,
         723: 1,
         609: 1,
         929: 1,
         944: 1,
         683: 1,
         425: 1,
         370: 1,
         304: 1,
         733: 1,
         444: 1,
         196: 1,
         828: 1,
         589: 1,
         935: 1,
         821: 1,
         811: 1,
 

In [17]:
outs = "The symphony concert was a truly remarkable experience that left the audience in awe. The orchestra, led by the talented conductor, performed with precision and passion, creating a harmonious and captivating atmosphere that filled the auditorium.\n\nFrom the moment the first note was played, it was clear that the orchestra had put in a tremendous amount of effort to prepare for the concert. Each instrument was played with skill and finesse, and the various sections of the orchestra worked together seamlessly to create a cohesive and beautiful sound.\n\nThe conductor was a true maestro, guiding the orchestra with a firm hand and a clear vision. Their gestures were precise and confident, and the orchestra responded with a level of precision and energy that was truly impressive.\n\nThe program was a mix of classical and contemporary pieces, and each one was performed with a level of artistry that was truly inspiring. The audience was treated to a range of emotions, from the soaring melodies of the classical pieces to the more modern and experimental sounds of the contemporary works.\n\nThe audience was captivated by the performance, and the applause at the end of each piece was a testament to the orchestra's skill and talent. The energy in the auditorium was palpable, and it was clear that everyone in attendance was thoroughly enjoying the experience.\n\nOverall, the symphony concert was a true masterpiece of music and performance. The orchestra's skill and passion were on full display, and the audience was treated to a truly unforgettable evening of music. It was a night that will be remembered for years to come. \n\n번역결과  \n교향곡 공연은 관객을 압도하는 놀라운 경험이었습니다. 뛰어난 지휘자로 이끄는 오케스트라가 정확하고 열정적으로 연주하여 공연장에 가득 찬 조화로운 및 매혹적인 분위기를 만들었습니다.\n첫 번째 음이 연주되자마자 오케스트라가 공연을 위해 엄청난 노력을 기울였다는 것이 분명했습니다. 각 악기는 기술과 섬세함으로 연주되었으며 오케스트라의 각 부분은 조화롭고 아름다운 소리를 만들기 위해 조화롭게 작동했습니다.\n지휘자는 명석한 손길과 명확한 비전으로 오케스트라를 지휘하는 진정한 마에스트로였습니다. 그들의 손짓은 정확하고 자신감이 넘쳤으며 오케스트라는 진정으로 놀라운 정확성과 에너지를 보여주었습니다.\n프로그램은 고전과 현대 작품의 혼합이었으며 각 작품은 진정으로 영감을 주는 예술적인 수준으로 연주되었습니다. 관객은 고전 작품의 날카로운 멜로디에서 현대 작품의 더 현대적이고 실험적인 소음까지 다양한 감정을 경험했습니다.\n각 작품의 끝마다 관객은 오케스트라의 기술과 재능을 찬양하는 박수 소리가 있었습니다. 공연장의 에너지는 분명히 느껴졌으며 모든 참석자가 즐기는 경험이었다는 것이 분명했습니다.\n오케스트라의 기술과 열정은 완벽한 작품의 음악 및 공연으로 나타났습니다. 오케스트라의 기술과 열정은 완벽하게 나타났으며 관객은 완전히 잊을 수 없는 음악의 밤을 즐길 수 있었습니다. 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로"
# outs2
outs_encode = mytokenizer.encode(outputs)
print(f"encode length: {len(outs_encode)}")

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [15]:
tokenizer.chat_template = tokenizer.chat_template.replace("<|eot_id|>", tokenizer.eos_token)
tokenizer.chat_template

"{{ '<|begin_of_text|>' }}{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ '<|start_header_id|>system<|end_header_id|>\n\n' + system_message + '<|end_of_text|>' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<|start_header_id|>user<|end_header_id|>\n\n' + content + '<|end_of_text|><|start_header_id|>assistant<|end_header_id|>\n\n' }}{% elif message['role'] == 'assistant' %}{{ content + '<|end_of_text|>' }}{% endif %}{% endfor %}"

In [17]:
eval_set

Dataset({
    features: ['instruction', 'generator', 'dataset', 'output'],
    num_rows: 805
})

In [23]:
def rename_generator(sample):
    sample['generator'] = f"{model_name}"
    return sample
eval_set = eval_set.map(rename_generator)

In [24]:
eval_set[:5]

{'instruction': ['What are the names of some famous actors that started their careers on Broadway?',
  'How did US states get their names?',
  "Hi, my sister and her girlfriends want me to play kickball with them. Can you explain how the game is played, so they don't take advantage of me?",
  'What is some cool music from the 1920s?',
  'How do I wrap a present neatly?'],
 'generator': ['Meta-Llama-3.1-8B',
  'Meta-Llama-3.1-8B',
  'Meta-Llama-3.1-8B',
  'Meta-Llama-3.1-8B',
  'Meta-Llama-3.1-8B'],
 'dataset': ['helpful_base',
  'helpful_base',
  'helpful_base',
  'helpful_base',
  'helpful_base'],
 'output': ['# What are the names of some famous actors that started their careers on Broadway?\n\n## Answers\n\n1. ### 1. Anonymous\n\n2 years ago\n\n2. ### 2. Anonymous\n\n2 years ago\n\n3. ### 3. Anonymous\n\n2 years ago\n\n4. ### 4. Anonymous\n\n2 years ago\n\n5. ### 5. Anonymous\n\n2 years ago\n\n6. ### 6. Anonymous\n\n2 years ago\n\n7. ### 7. Anonymous\n\n2 years ago\n\n8. ### 8. Anony

In [30]:
## save data
import json
export_dataset = eval_set
export_data_list = [dict(row) for row in export_dataset]
print(f"export data length {len(export_data_list)}")
with open(f'./data/{model_name}.json', 'w', encoding='utf-8') as f:  
    json.dump(export_data_list, f, ensure_ascii=False, indent=4)  

export data length 805


In [ ]:
gen_kwargs_vllm = {
    "max_tokens": 2048,
    "top_p": 0.9,
    "top_k": 50,
    "temperature": 0.0,
    "repetition_penalty": 1.0,
}
# gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|end_of_text|>")]
sampling_params = SamplingParams(**gen_kwargs_vllm)
sampling_params

In [14]:
eval_set.to_json(f"{model_name}.jsonl", batch_size=128, num_proc=8)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Creating json from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

2874764

In [4]:
outs = "The symphony concert was a truly remarkable experience that left the audience in awe. The orchestra, led by the talented conductor, performed with precision and passion, creating a harmonious and captivating atmosphere that filled the auditorium.\n\nFrom the moment the first note was played, it was clear that the orchestra had put in a tremendous amount of effort to prepare for the concert. Each instrument was played with skill and finesse, and the various sections of the orchestra worked together seamlessly to create a cohesive and beautiful sound.\n\nThe conductor was a true maestro, guiding the orchestra with a firm hand and a clear vision. Their gestures were precise and confident, and the orchestra responded with a level of precision and energy that was truly impressive.\n\nThe program was a mix of classical and contemporary pieces, and each one was performed with a level of artistry that was truly inspiring. The audience was treated to a range of emotions, from the soaring melodies of the classical pieces to the more modern and experimental sounds of the contemporary works.\n\nThe audience was captivated by the performance, and the applause at the end of each piece was a testament to the orchestra's skill and talent. The energy in the auditorium was palpable, and it was clear that everyone in attendance was thoroughly enjoying the experience.\n\nOverall, the symphony concert was a true masterpiece of music and performance. The orchestra's skill and passion were on full display, and the audience was treated to a truly unforgettable evening of music. It was a night that will be remembered for years to come. \n\n번역결과  \n교향곡 공연은 관객을 압도하는 놀라운 경험이었습니다. 뛰어난 지휘자로 이끄는 오케스트라가 정확하고 열정적으로 연주하여 공연장에 가득 찬 조화로운 및 매혹적인 분위기를 만들었습니다.\n첫 번째 음이 연주되자마자 오케스트라가 공연을 위해 엄청난 노력을 기울였다는 것이 분명했습니다. 각 악기는 기술과 섬세함으로 연주되었으며 오케스트라의 각 부분은 조화롭고 아름다운 소리를 만들기 위해 조화롭게 작동했습니다.\n지휘자는 명석한 손길과 명확한 비전으로 오케스트라를 지휘하는 진정한 마에스트로였습니다. 그들의 손짓은 정확하고 자신감이 넘쳤으며 오케스트라는 진정으로 놀라운 정확성과 에너지를 보여주었습니다.\n프로그램은 고전과 현대 작품의 혼합이었으며 각 작품은 진정으로 영감을 주는 예술적인 수준으로 연주되었습니다. 관객은 고전 작품의 날카로운 멜로디에서 현대 작품의 더 현대적이고 실험적인 소음까지 다양한 감정을 경험했습니다.\n각 작품의 끝마다 관객은 오케스트라의 기술과 재능을 찬양하는 박수 소리가 있었습니다. 공연장의 에너지는 분명히 느껴졌으며 모든 참석자가 즐기는 경험이었다는 것이 분명했습니다.\n오케스트라의 기술과 열정은 완벽한 작품의 음악 및 공연으로 나타났습니다. 오케스트라의 기술과 열정은 완벽하게 나타났으며 관객은 완전히 잊을 수 없는 음악의 밤을 즐길 수 있었습니다. 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로 인해 오케스트라의 연주로"
outs_encode = tokenizer.encode(outs)

NameError: name 'tokenizer' is not defined

In [8]:
# from vllm import LLM, SamplingParams

# # Sample prompts.
# prompts = [
#     "Hello, my name is",
#     "The president of the United States is",
#     "The capital of France is",
#     "The future of AI is",
# ]
# # Create a sampling params object.
# sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

# # Create an LLM.
# llm = LLM(model="/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface/Meta-Llama-3.1-8B-Instruct")
# # Generate texts from the prompts. The output is a list of RequestOutput objects
# # that contain the prompt, generated text, and other information.
# messages=[
# {"role": "user", "content": "Hello!"}
# ]
# tokenizer = llm.get_tokenizer()
# messages = tokenizer.apply_chat_template(messages, tokenize=False)

# outputs = llm.generate(prompts, sampling_params)
# # Print the outputs.
# for output in outputs:
#     prompt = output.prompt
#     generated_text = output.outputs[0].text
#     print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

In [4]:
eval_set
def convert_to_message(example):  
    messages = [{"role": "user", "content": example["instruction"]}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
eval_set = eval_set.map(convert_to_message)
# eval_set['messages']
outputs = llm.generate(eval_set['messages'], sampling_params)
outputs_text = [x.outputs[0].text for x in outputs]
eval_set = eval_set.remove_columns(["output"])  # Remove the existing 'output' column if it exists  
eval_set = eval_set.remove_columns(["messages"])
eval_set = eval_set.add_column("output", outputs_text)  

Map: 100%|██████████| 805/805 [00:00<00:00, 11922.37 examples/s]


['<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nWhat are the names of some famous actors that started their careers on Broadway?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n',
 '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nHow did US states get their names?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n',
 "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nHi, my sister and her girlfriends want me to play kickball with them. Can you explain how the game is played, so they don't take advantage of me?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
 '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nWhat is some cool music from the 1920s?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n',
 '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nHow do I wrap a present neatly?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n',
 '<|begin_of_text|><|start_header_id|>user<|en

In [5]:
outputs = llm.generate(eval_set['messages'], sampling_params)
outputs_text = [x.outputs[0].text for x in outputs]
eval_set = eval_set.remove_columns(["output"])  # Remove the existing 'output' column if it exists  
eval_set = eval_set.remove_columns(["messages"])
eval_set = eval_set.add_column("output", outputs_text)  

Processed prompts:   0%|          | 0/805 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 805/805 [00:09<00:00, 83.07it/s, est. speed input: 3862.30 toks/s, output: 1283.79 toks/s] 


In [14]:
# outputs_text

In [15]:
eval_set = eval_set.remove_columns(["output"])  # Remove the existing 'output' column if it exists  
eval_set = eval_set.remove_columns(["messages"])
eval_set = eval_set.add_column("output", outputs_text)  

In [16]:
eval_set[:5]

{'instruction': ['What are the names of some famous actors that started their careers on Broadway?',
  'How did US states get their names?',
  "Hi, my sister and her girlfriends want me to play kickball with them. Can you explain how the game is played, so they don't take advantage of me?",
  'What is some cool music from the 1920s?',
  'How do I wrap a present neatly?'],
 'generator': ['text_davinci_003',
  'text_davinci_003',
  'text_davinci_003',
  'text_davinci_003',
  'text_davinci_003'],
 'dataset': ['helpful_base',
  'helpful_base',
  'helpful_base',
  'helpful_base',
  'helpful_base'],
 'messages': ['<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nWhat are the names of some famous actors that started their careers on Broadway?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n',
  '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nHow did US states get their names?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n',
  "<|begin_of_text|><